## Overview

This notebook is part of a larger project aimed at analyzing the Ames Housing dataset and building predictive models for housing prices. The project is organized into several sections, each focusing on a different aspect of the data analysis and modeling process. The primary goal is to provide a comprehensive understanding of the factors influencing house prices and to develop robust predictive models.

## Purpose of This Notebook

The purpose of this notebook is to perform data preparation and preprocessing to ensure the dataset is clean and ready for modeling. This involves handling missing values, encoding categorical variables, standardizing numerical features, and removing outliers. These steps are crucial for improving the performance and reliability of the predictive models.

**Data Preparation and Preprocessing**

**1. Handling Missing Values:**

- Identify columns with missing values and their counts.
- Impute missing values for numerical columns using the median strategy.
- Impute missing values for categorical columns using the most frequent value strategy.
- Ensure no missing values remain after imputation.

**2. Encoding Categorical Variables:**

- Convert categorical variables into dummy/indicator variables using one-hot encoding.

**3. Standardizing Numerical Features:**

- Standardize all numerical features except 'SalePrice' using StandardScaler.
- Scale 'SalePrice' separately to facilitate easier inversion of the scaling during final reporting.

**4. Removing Outliers:**

- Remove outliers based on GrLivArea to ensure the dataset is not skewed by extreme values.
- Specifically, rows where GrLivArea is greater than 4000 square feet are dropped.

In [14]:
# Load necessary libraries
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.impute import SimpleImputer

In [15]:
# Load the Ames dataset
file_path = '../data/AmesData.csv'
ames_df = pd.read_csv(file_path)

# Identify columns with missing values
missing_values = ames_df.isnull().sum()
print("\nColumns with missing values and their counts before preprocessing:")
print(missing_values[missing_values > 0])


Columns with missing values and their counts before preprocessing:
Alley           2593
MasVnrType      1629
MasVnrArea        21
BsmtQual          64
BsmtCond          64
BsmtExposure      67
BsmtFinType1      64
BsmtFinType2      65
Electrical         1
FireplaceQu     1277
PoolQC          2756
Fence           2227
MiscFeature     2671
dtype: int64


In [16]:
# Load the Ames dataset
file_path = '../data/AmesData.csv'
ames_df = pd.read_csv(file_path)

# Specifically replace blanks in 'MasVnrArea' with 0
ames_df.replace({'MasVnrArea': 'nan'}, 0, inplace=True)

# Identify columns with missing values
missing_values = ames_df.isnull().sum()
print("\nColumns with missing values and their counts before preprocessing:")
print(missing_values[missing_values > 0])

# Handle missing values for numerical columns
num_imputer = SimpleImputer(strategy='median')
num_cols_with_missing = ames_df.select_dtypes(include=[np.number]).columns[ames_df.select_dtypes(include=[np.number]).isnull().any()].tolist()
ames_df[num_cols_with_missing] = num_imputer.fit_transform(ames_df[num_cols_with_missing])

# Handle missing values for categorical columns
cat_imputer = SimpleImputer(strategy='most_frequent')
cat_cols_with_missing = ames_df.select_dtypes(exclude=[np.number]).columns[ames_df.select_dtypes(exclude=[np.number]).isnull().any()].tolist()
ames_df[cat_cols_with_missing] = cat_imputer.fit_transform(ames_df[cat_cols_with_missing])

# Ensure no missing values remain
print("\nColumns with missing values after imputation and their counts:")
print(ames_df.isnull().sum()[ames_df.isnull().sum() > 0])

# Convert boolean columns to integer
bool_cols = ames_df.select_dtypes(include=[bool]).columns.tolist()
ames_df[bool_cols] = ames_df[bool_cols].astype(int)

# Check the number of rows and columns in the dataset before outlier removal
num_rows, num_columns = ames_df.shape
print(f"\nThe dataset contains {num_rows} rows and {num_columns} columns before outlier removal.")

# Remove outliers based on 'GrLivArea'
initial_row_count = ames_df.shape[0]
ames_df = ames_df.drop(ames_df[ames_df['GrLivArea'] > 4000].index)
final_row_count = ames_df.shape[0]
print(f"Number of rows removed based on 'GrLivArea' > 4000: {initial_row_count - final_row_count}")

# Check the number of rows and columns in the dataset after outlier removal
num_rows, num_columns = ames_df.shape
print(f"\nThe dataset contains {num_rows} rows and {num_columns} columns after outlier removal.")

# Encoding Categorical Variables
ames_df = pd.get_dummies(ames_df, drop_first=True)

# Check the number of rows and columns in the dataset after encoding
num_rows, num_columns = ames_df.shape
print(f"\nThe dataset contains {num_rows} rows and {num_columns} columns after encoding.")

## Note: This scaling didn't work, I couldn't figure out how to invert the scaling after prediction. 

# Standardize all numerical features
# scaler = StandardScaler()
# num_features = ames_df.select_dtypes(include=[np.number]).columns
# ames_df[num_features] = scaler.fit_transform(ames_df[num_features])

#Alternative Approach:
# Standardize all numerical features except 'SalePrice'
scaler = StandardScaler()
num_features = ames_df.select_dtypes(include=[np.number]).columns
num_features = num_features.drop('SalePrice')
ames_df[num_features] = scaler.fit_transform(ames_df[num_features])

# Scale 'SalePrice' separately
saleprice_scaler = StandardScaler()
ames_df['SalePrice'] = saleprice_scaler.fit_transform(ames_df['SalePrice'].values.reshape(-1, 1))

# Check the number of rows and columns in the dataset after scaling
num_rows, num_columns = ames_df.shape
print(f"\nThe dataset contains {num_rows} rows and {num_columns} columns after scaling.")


Columns with missing values and their counts before preprocessing:
Alley           2593
MasVnrType      1629
MasVnrArea        21
BsmtQual          64
BsmtCond          64
BsmtExposure      67
BsmtFinType1      64
BsmtFinType2      65
Electrical         1
FireplaceQu     1277
PoolQC          2756
Fence           2227
MiscFeature     2671
dtype: int64

Columns with missing values after imputation and their counts:
Series([], dtype: int64)

The dataset contains 2769 rows and 80 columns before outlier removal.
Number of rows removed based on 'GrLivArea' > 4000: 5

The dataset contains 2764 rows and 80 columns after outlier removal.

The dataset contains 2764 rows and 256 columns after encoding.

The dataset contains 2764 rows and 256 columns after scaling.
